In [1]:
# Imports
import pyarrow.parquet as pq
import pandas as pd
import numpy as np
from tqdm.notebook import tqdm
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split
import cv2, os, gc, datetime, json

# os.environ['CUDA_VISIBLE_DEVICES'] = '-1'

import tensorflow as tf
gpu_devices = tf.config.experimental.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(gpu_devices[0], True)

# Keras imports
from tensorflow.keras import backend as K
from tensorflow.keras.models import Model, load_model, model_from_json
from tensorflow.keras.models import clone_model
from tensorflow.keras.layers import Dense, Conv2D, Flatten, GlobalAveragePooling2D, MaxPool2D, Dropout, BatchNormalization, Input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ReduceLROnPlateau
from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2
from tensorflow.keras.applications.resnet_v2 import ResNet50V2, ResNet152V2
from tensorflow.keras.applications.resnet import ResNet50, ResNet152
from tensorflow.keras.applications.inception_resnet_v2 import InceptionResNetV2
from tensorflow.keras.applications.xception import Xception
from tensorflow.keras.applications.nasnet import NASNetLarge, NASNetMobile
from tensorflow.keras.metrics import Accuracy, Precision, Recall
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, TensorBoard, ModelCheckpoint
from tensorflow.keras.optimizers import Adam, SGD, RMSprop, Adadelta, Nadam, Adagrad

In [18]:
test_path = 'data/images/test/'

HEIGHT = 137
WIDTH = 236
SIZE = 128
BATCH_SIZE = 64

MODEL_PATH = "saved_model/model_inception_resnet.json"
WEIGHTS_PATH = "saved_model/model_inception_resnet.h5"

In [3]:
test_csv_path = "./data/test.csv"
df = pd.read_csv(test_csv_path)

In [6]:
with open(MODEL_PATH, 'r') as f:
    model_config = f.read()
model = model_from_json(model_config)
model.load_weights(WEIGHTS_PATH)

In [9]:
# Model Summary
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 128, 128, 1) 0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 128, 128, 3)  78          input_1[0][0]                    
__________________________________________________________________________________________________
inception_resnet_v2 (Model)     multiple             54336736    conv2d[0][0]                     
__________________________________________________________________________________________________
global_average_pooling2d (Globa (None, 1536)         0           inception_resnet_v2[1][0]        
______________________________________________________________________________________________

In [58]:
df['target'] = 1
for index in tqdm(range(len(df))):
    filename = df.iloc[index]['image_id']
    path = os.path.join(test_path, f"{filename}.jpg")
    image = cv2.resize(cv2.imread(path, 0), (SIZE, SIZE), cv2.INTER_AREA)/255
    image = np.expand_dims(image, axis=2)
    root, vowel, consonant = model.predict(np.expand_dims(image, axis=0))
    if df.iloc[index]['component'] == 'consonant_diacritic':
        df.set_value(index, 'target', np.argmax(consonant))
    if df.iloc[index]['component'] == 'grapheme_root':
        df.set_value(index, 'target', np.argmax(root))
    if df.iloc[index]['component'] == 'vowel_diacritic':
        df.set_value(index, 'target', np.argmax(vowel))

/home/ani/miniconda3/envs/bengocr/lib/python3.7/site-packages/ipykernel_launcher.py:9: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  if __name__ == '__main__':
/home/ani/miniconda3/envs/bengocr/lib/python3.7/site-packages/ipykernel_launcher.py:11: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  # This is added back by InteractiveShellApp.init_path()
/home/ani/miniconda3/envs/bengocr/lib/python3.7/site-packages/ipykernel_launcher.py:13: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  del sys.path[0]


In [61]:
df.drop(['image_id', 'component'], axis=1, inplace=True)

In [63]:
df.to_csv('submissions.csv', index=False)